In [1]:
import tseriesRoutines as routines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import ceil, sqrt
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
from keras.models import load_model

Using TensorFlow backend.


In [2]:
def forecastModelVal(mongoid, conn, cursor, folder, salescluster=None):
    '''
    '''
    # clusters
    listOfClusters = (1, 2, 3)
    # list of old id-s
    listOfId = ('5aa2ad7735d6d34b0032a795', '5aa39533ae1f941be7165ecd',
               '5a93e8768cbad97881597597', '5a95d7ae35d6d33d3fea56ff',
               '5aa2c35e35d6d34b0032a796', '5a92474635d6d32207bcd343')
    # models' path
    dictOfModels = {
            # 1st special
            'sp1': folder + 'forecast_nn_5aa2ad7735d6d34b0032a795_bigrumodel.h5',
            'sp2': folder + 'forecast_nn_5aa39533ae1f941be7165ecd_flatmodel.h5',
            # cluster 3
            'c3': folder + 'forecast_nn_5a93e8768cbad97881597597_grudropmodel.h5',
            'c3_2': folder + 'forecast_nn_5a95d7ae35d6d33d3fea56ff_grumodel.h5',
            # cluster 1
            'c1': folder + 'forecast_nn_5aa2c35e35d6d34b0032a796_bigrumodel.h5',
            # cluster 2
            'c2': folder + 'forecast_nn_5a92474635d6d32207bcd343_flatmodel.h5'
            }

#    if mongoid in listOfId:
    if mongoid == '5aa2ad7735d6d34b0032a795' or '5b32126eb8a9f70142f6a766':
        model = load_model(dictOfModels['sp1'])
    elif mongoid == '5aa39533ae1f941be7165ecd' or '5b322e9cb8a9f70142f6a767': 
        model = load_model(dictOfModels['sp2'])
    elif mongoid == '5a93e8768cbad97881597597' or '5b324265b8a9f70142f6a768': 
        model = load_model(dictOfModels['c3'])
    elif mongoid == '5a95d7ae35d6d33d3fea56ff' or '5b32509bb8a9f70142f6a769': 
        model = load_model(dictOfModels['c3_2'])
    elif mongoid == '5aa2c35e35d6d34b0032a796' or '5b3277a3b8a9f70142f6a76b': 
        model = load_model(dictOfModels['c1'])
    elif mongoid == '5a92474635d6d32207bcd343' or '5b36082535d6d30034572c71': 
        model = load_model(dictOfModels['c2'])
    else:
        if salescluster in listOfClusters:
            if salescluster == 1:
                model = load_model(dictOfModels['c1'])
            elif salescluster == 2:
                model = load_model(dictOfModels['c2'])
            elif salescluster == 3:
                model = load_model(dictOfModels['c3'])
        else:
            print('Enter ID\'s Sales Cluster')

    return model

In [3]:
def splitDataNN(df, n_in=1, n_out=1, scale=True):
    '''
    unlike routines.splitDataNN, this splitDataNN doesn't split data 
    into train and test set data
    '''
    dftrain = routines.series_to_supervised(df, n_in=n_in, n_out=n_out)
    # specific to this case
    dftrain = dftrain.drop(dftrain.columns[[4, 5]], axis=1)
    values = dftrain.values

    if scale:
        scaler = MinMaxScaler()
        values = scaler.fit_transform(values)
    else:
        pass

    # training data
    X, y = values[:, :-1], values[:, -1]
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    return X, y, dftrain, scaler

In [4]:
# make connection to sqlite db
conn = sqlite3.connect('validasi.db')
c = conn.cursor()

# enable foreign keys
c.execute("PRAGMA foreign_keys = ON")
conn.commit()

# Folder contains saved models
folder = './training/'
# IDs to validate models
listOfId = ('5aa2ad7735d6d34b0032a795', '5aa39533ae1f941be7165ecd',
           '5a93e8768cbad97881597597', '5a95d7ae35d6d33d3fea56ff',
           '5aa2c35e35d6d34b0032a796', '5a92474635d6d32207bcd343')

In [32]:
product = routines.genDataVal('5aa2ad7735d6d34b0032a795', conn, c, impute=False, freq='daily')
X, y, dftrain, scaler = splitDataNN(product)

In [33]:
# load model
model = forecastModelVal('5aa2ad7735d6d34b0032a795', conn, c, folder=folder, salescluster=None)


In [35]:
# make prediction
ypred = model.predict(X)

In [36]:
ypred.shape

(116, 1)

In [37]:
    # reshape X
    X = X.reshape((X.shape[0], X.shape[2]))


In [38]:
X.shape

(116, 3)

In [39]:
X

array([[0.42857143, 1.        , 0.83333333],
       [0.42857143, 1.        , 1.        ],
       [0.        , 0.        , 0.5       ],
       [0.71428571, 0.96      , 0.9       ],
       [0.14285714, 1.        , 1.        ],
       [0.85714286, 0.96666667, 0.91666667],
       [0.28571429, 0.9       , 1.        ],
       [0.14285714, 1.        , 1.        ],
       [0.42857143, 1.        , 0.83333333],
       [0.42857143, 1.        , 0.66666667],
       [0.42857143, 1.        , 0.66666667],
       [0.42857143, 1.        , 1.        ],
       [0.42857143, 0.93333333, 0.83333333],
       [0.42857143, 1.        , 1.        ],
       [0.42857143, 1.        , 0.83333333],
       [0.42857143, 1.        , 1.        ],
       [0.        , 0.        , 0.5       ],
       [0.42857143, 1.        , 1.        ],
       [0.85714286, 0.96666667, 0.91666667],
       [0.        , 0.        , 0.5       ],
       [0.42857143, 0.93333333, 1.        ],
       [0.42857143, 1.        , 0.83333333],
       [0.

In [40]:
#invert scaling for forecast
inv_ypred = np.concatenate((X[:, :], ypred), axis=1)

In [41]:
inv_ypred.shape

(116, 4)

In [42]:
inv_ypred[0]

array([0.42857143, 1.        , 0.83333333, 0.24428488])

In [43]:
dftrain.shape

(116, 4)

In [44]:
dftrain.head()

,var1(t-1),var2(t-1),var3(t-1),var1(t)
date,,,,
2018-03-03,3.0,5.0,0.666667,3
2018-03-04,3.0,5.0,1.000000,0
2018-03-05,0.0,0.0,0.000000,5
2018-03-06,5.0,4.8,0.800000,1
2018-03-07,1.0,5.0,1.000000,6


In [45]:
inv_ypred = scaler.inverse_transform(inv_ypred)

In [46]:
inv_ypred = inv_ypred[:, -1]

In [47]:
inv_ypred = np.ceil(inv_ypred)

In [48]:
y = y.reshape((len(y), 1))

In [49]:
y

array([[0.42857143],
       [0.        ],
       [0.71428571],
       [0.14285714],
       [0.85714286],
       [0.28571429],
       [0.14285714],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.        ],
       [0.42857143],
       [0.85714286],
       [0.        ],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.57142857],
       [0.71428571],
       [0.85714286],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.        ],
       [0.42857143],
       [0.42857143],
       [0.85714286],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.42857143],
       [0.71428571],
       [0.28571429],
       [0.71428571],
       [0.42857143],
       [0.85714286],
       [0.        ],
       [0.42857143],
       [0.14285714],
       [0.28571429],
       [0.71428571],
       [0.285

In [50]:
y.shape

(116, 1)

In [51]:
X.shape

(116, 3)

In [52]:
inv_y = np.concatenate((X[:, :], y), axis=1)

In [53]:
inv_y = scaler.inverse_transform(inv_y)

In [54]:
inv_y

array([[ 3.        ,  5.        ,  0.66666667,  3.        ],
       [ 3.        ,  5.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  5.        ],
       [ 5.        ,  4.8       ,  0.8       ,  1.        ],
       [ 1.        ,  5.        ,  1.        ,  6.        ],
       [ 6.        ,  4.83333333,  0.83333333,  2.        ],
       [ 2.        ,  4.5       ,  1.        ,  1.        ],
       [ 1.        ,  5.        ,  1.        ,  3.        ],
       [ 3.        ,  5.        ,  0.66666667,  3.        ],
       [ 3.        ,  5.        ,  0.33333333,  3.        ],
       [ 3.        ,  5.        ,  0.33333333,  3.        ],
       [ 3.        ,  5.        ,  1.        ,  3.        ],
       [ 3.        ,  4.66666667,  0.66666667,  3.        ],
       [ 3.        ,  5.        ,  1.        ,  3.        ],
       [ 3.        ,  5.        ,  0.66666667,  3.        ],
       [ 3.        ,  5.        ,  1.        ,  0.        ],
       [ 0.        ,  0.

In [55]:
inv_y = inv_y[:, -1]

In [56]:
np.sqrt(mean_squared_error(y_pred=inv_ypred, y_true=inv_y))

1.7419766814227708

In [57]:
mean_absolute_error(y_pred=inv_ypred, y_true=inv_y)

1.3793103448275863

In [58]:
listOfId2 = ('5b32126eb8a9f70142f6a766', '5b322e9cb8a9f70142f6a767',
        '5b324265b8a9f70142f6a768', '5b32509bb8a9f70142f6a769',
        '5b3277a3b8a9f70142f6a76b', '5b36082535d6d30034572c71')

In [59]:
for i in listOfId2:
    # Generate dataframe and X & y
    product = routines.genDataVal(i, conn, c, impute=False, freq='daily')
    X, y, dftrain, scaler = splitDataNN(product)

    # load model
    model = forecastModelVal(i, conn, c, folder=folder, salescluster=None)
    # make prediction
    ypred = model.predict(X)
    # reshape X
    X = X.reshape((X.shape[0], X.shape[2]))
    #invert scaling for forecast
    inv_ypred = np.concatenate((X[:, :], ypred), axis=1)
    inv_ypred = scaler.inverse_transform(inv_ypred)
    inv_ypred = inv_ypred[:, -1]
    # invert scaling for actual
    y = y.reshape((len(y), 1))
    inv_y = np.concatenate((X[:, :], y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:, -1]

    # RMSE
    rmse = np.sqrt(mean_squared_error(y_pred=inv_ypred, y_true=inv_y))
    # MAE
    mae = mean_absolute_error(y_pred=inv_ypred, y_true=inv_y)

    print('MongoDB ID: {0}'.format(i))
    print('Validasi RMSE: {0:.5f}'.format(rmse))
    print('Validasi MAE: {0:.5f}'.format(mae))

MongoDB ID: 5b32126eb8a9f70142f6a766
Validasi RMSE: 1.91848
Validasi MAE: 1.54137
MongoDB ID: 5b322e9cb8a9f70142f6a767
Validasi RMSE: 1.11741
Validasi MAE: 0.97843
MongoDB ID: 5b324265b8a9f70142f6a768
Validasi RMSE: 3.60600
Validasi MAE: 3.18236
MongoDB ID: 5b32509bb8a9f70142f6a769
Validasi RMSE: 1.39227
Validasi MAE: 1.20846
MongoDB ID: 5b3277a3b8a9f70142f6a76b
Validasi RMSE: 5.57222
Validasi MAE: 5.01857
MongoDB ID: 5b36082535d6d30034572c71
Validasi RMSE: 0.49319
Validasi MAE: 0.45304
